In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [24]:
df = pd.read_csv("HR_comma_sep.csv")
df.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


**From last week's data analysis**
<ol>
    <li>Satisfaction Level</li>
    <li>Average Monthly Hours</li>
    <li>Promotion Last 5 Years</li>
    <li>Salary</li>
</ol>

In [3]:
subdf = df[['satisfaction_level','average_montly_hours','promotion_last_5years','salary']]
# subdf.describe()
subdf.salary.unique()
# subdf

array(['low', 'medium', 'high'], dtype=object)

**Tackle salary dummy variable**

In [4]:
from sklearn.preprocessing import LabelEncoder
le_salary = LabelEncoder()
subdf["salary"] = le_salary.fit_transform(subdf["salary"])
subdf["salary"].unique()

/tmp/ipykernel_10372/1469059559.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf["salary"] = le_salary.fit_transform(subdf["salary"])


array([1, 2, 0])

In [5]:
X = subdf
X.head()

,satisfaction_level,average_montly_hours,promotion_last_5years,salary
0,0.38,157,0,1
1,0.80,262,0,2
2,0.11,272,0,2
3,0.72,223,0,1
4,0.37,159,0,1


In [6]:
y = df.left

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.3)

In [8]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [9]:
model.fit(X_train, y_train)

LogisticRegression()

In [10]:
model.predict(X_test)

array([0, 0, 1, ..., 0, 0, 0])

**Accuracy of the model**

In [11]:
model.score(X_test,y_test)

0.7716190476190476

In [12]:
import pickle

In [13]:
data = {"model": model, "le_salary": le_salary}
with open('HR_model.pkl', 'wb') as file:
  pickle.dump(data, file)

In [14]:
with open('HR_model.pkl', 'rb') as file:
  data = pickle.load(file)

model = data["model"]
le_salary = data["le_salary"]

In [15]:
model.predict(X_test)

array([0, 0, 1, ..., 0, 0, 0])

In [16]:
model.score(X_test,y_test)

0.7716190476190476

In [17]:
import numpy as np
X_test1 = np.array([[0, 140, 0, "low"]])
X_test1

array([['0', '140', '0', 'low']], dtype='<U21')

In [18]:
X_test1[:,-1] = le_salary.transform(X_test1[:,-1])
X_test1

array([['0', '140', '0', '1']], dtype='<U21')

In [19]:
X_test1 = X_test1.astype(float)

In [20]:
model.predict(X_test1)

/home/knust/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([1])

In [21]:
X_test1 = pd.DataFrame({
    "satisfaction_level":[0],
    "average_montly_hours" : [140],
    "promotion_last_5years": [0],
    "salary": ["low"]
})
X_test1

,satisfaction_level,average_montly_hours,promotion_last_5years,salary
0,0,140,0,low


In [22]:
X_test1.salary = le_salary.transform(X_test1.salary)
X_test1

,satisfaction_level,average_montly_hours,promotion_last_5years,salary
0,0,140,0,1


In [23]:
model.predict(X_test1)

array([1])